# Sleaper API Relative Endpoints

In [0]:
# fetch parameters
par_league_id = dbutils.widgets.get("LEAGUE_ID")
par_matchup_week = int( dbutils.widgets.get("MATCHUP_WEEK") )
par_year = dbutils.widgets.get("YEAR")

In [0]:
import requests
import json
from datetime import datetime
tgt_dir = '/mnt/databricks/sleeper/stg'

_TIMESTAMP = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

def add_metadata(array_data):
    for row in array_data:
        row['_year'] = int(par_year)
        row['_league_id'] = par_league_id
        row['_matchup_week'] = par_matchup_week
        row['_ingested_ts'] = _TIMESTAMP
    return array_data

## /v1/league/*

In [0]:
if par_matchup_week > 0:

    endpoints = ['rosters', f'matchups/{par_matchup_week}', 'users', f'transactions/{par_matchup_week}']

    for endpoint in endpoints:

        # Fetch data from the API
        response = requests.get(f"https://api.sleeper.app/v1/league/{par_league_id}/{endpoint}")
        data = response.json()

        # add metadata
        data = add_metadata(data)
        # Write JSON data to a volume with a unique timestamp
        endpoint_dir = endpoint.split('/')[0]

        dbutils.fs.put(f"{tgt_dir}/{endpoint_dir}/{par_year}/{endpoint_dir}_{par_league_id}_{par_matchup_week}_{timestamp}.json", json.dumps(data), overwrite=True)

## v1/league

In [0]:
if par_matchup_week > 0:
    
    response = requests.get(f"https://api.sleeper.app/v1/league/{par_league_id}/")
    data = response.json()

    # add metadata
    data = add_metadata([data])
    # Write JSON data to a volume with a unique timestamp

    dbutils.fs.put(f"{tgt_dir}/league/{par_year}/league_{par_league_id}_{par_matchup_week}_{timestamp}.json", json.dumps(data), overwrite=True)

## v1/drafts

In [0]:
if par_matchup_week == 0:

    league_endpoints = ['drafts']

    for endpoint in league_endpoints:

        # Fetch data from the API
        response = requests.get(f"https://api.sleeper.app/v1/league/{par_league_id}/{endpoint}")
        data = response.json()

        # add metadata
        data = add_metadata(data)
        # Write JSON data to a volume with a unique timestamp
        endpoint_dir = endpoint.split('/')[0]

        dbutils.fs.put(f"{tgt_dir}/{endpoint_dir}/{par_year}/{endpoint_dir}_{par_league_id}_{par_matchup_week}_{timestamp}.json", json.dumps(data), overwrite=True)

        latest_draft_id = data[0]['draft_id']

        response = requests.get(f"https://api.sleeper.app/v1/draft/{latest_draft_id}/picks")
        data = response.json()

        data = add_metadata(data)

        dbutils.fs.put(f"{tgt_dir}/{endpoint_dir + '_picks'}/{par_year}/{endpoint_dir + '_picks'}_{par_league_id}_{par_matchup_week}_{timestamp}.json", json.dumps(data), overwrite=True)

        endpoints = ['users']

        for endpoint in endpoints:

            # Fetch data from the API
            response = requests.get(f"https://api.sleeper.app/v1/league/{par_league_id}/{endpoint}")
            data = response.json()

            # add metadata
            data = add_metadata(data)
            # Write JSON data to a volume with a unique timestamp
            endpoint_dir = endpoint.split('/')[0]

            dbutils.fs.put(f"{tgt_dir}/{endpoint_dir}/{par_year}/{endpoint_dir}_{par_league_id}_{par_matchup_week}_{timestamp}.json", json.dumps(data), overwrite=True)


## v1/players

In [0]:
if par_matchup_week > 0:
    player_api_response = requests.get(f'https://api.sleeper.app/v1/players/nfl')
    player_api_raw_json = json.loads(player_api_response.text)

    player_api_json = [v for k, v in player_api_raw_json.items() if isinstance(v, dict) and 'player_id' in v.keys()]

    player_api_json = add_metadata(player_api_json)
    dbutils.fs.put(f"{tgt_dir}/players/{par_year}/players_{par_league_id}_{par_matchup_week}_{timestamp}.json", json.dumps(player_api_json), overwrite=True)